# Test CXR Report Gen Plugin

This Jupyter notebook demonstrate how to test the cxr_report_gen.py plugin. Running this notebook allows the code to be tested without deployment.

In [ ]:
import os
import sys

CUR_DIR = os.path.abspath("")
ROOT_DIR = os.path.dirname(os.path.dirname(CUR_DIR))
SRC_DIR = os.path.join(ROOT_DIR, 'src')

sys.path.append(SRC_DIR)
print(f"Added {SRC_DIR} to sys.path")

In [ ]:
from config import setup_logging
from dotenv import load_dotenv

setup_logging()
load_dotenv(os.path.join(SRC_DIR, '.env'))

In [ ]:
from app import create_app_context


app_ctx = create_app_context()

In [ ]:
from config import load_agent_config
from data_models.chat_context import ChatContext
from data_models.plugin_configuration import PluginConfiguration
from scenarios.default.tools.cxr_report_gen import create_plugin
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion


agent_config = load_agent_config("default")
radiology_agent_config = [config for config in agent_config if config["name"] == "Radiology"][0]
url = radiology_agent_config["hls_model_endpoint"].get("cxr_report_gen")
patient_id = "patient_4"
filename = "x-ray.png"

kernel = Kernel()
kernel.add_service(
    AzureChatCompletion(
        service_id="default",
        deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        api_version="2024-10-21"
    )
)

chat_ctx = ChatContext("test-conversation-id")
chat_ctx.patient_id = patient_id
chat_ctx.patient_data = await app_ctx.data_access.image_accessor.get_metadata_list(patient_id)

plugin_config = PluginConfiguration(
    kernel=kernel,
    chat_ctx=chat_ctx,
    agent_config=radiology_agent_config,
    data_access=app_ctx.data_access,
    azureml_token_provider=app_ctx.azureml_token_provider)

cxr_report_gen_plugin = create_plugin(plugin_config)
await cxr_report_gen_plugin.generate_findings(patient_id, filename, "pneumonia")